#### Step 1: Import gurobipy module

In [1]:
import gurobipy as gp
from gurobipy import GRB

#### Step 2: Define your model

In [2]:
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-08


#### Step 4: Define your parameters

In [3]:
import random
import math
#maximum battery life
T = random.randrange(20,30)

#Number of nodes in the network
N = 18

#
D = 18

#cost
c = random.randrange(3,5)

#penalty cost increase
p = random.uniform(1.5, 2.0)

#discount time limit
H = random.randrange(20,30)

#Big M
M = 10^6

t = {}

#tij time between nodes
arcs = [(1,2), (1,3), 
        (2,4), (2,5), (2, 6), 
        (3, 6), 
        (4, 5), (4, 9), 
        (5,8),
        (6, 7),(6, 8),
        (7, 8),(7, 11),
        (8, 9),(8, 10),(8, 11),
        (9, 12),(9, 13),
        (10,12),
        (11,12),(11,14),
        (12,13),(12,14),
        (13,15),(14,16),
        (14,17),
        (15,16),(15,18),
        (16,17),(16,18),
        (17,18)]

for link in arcs:
    i = link[0]
    j = link[1]
    t[i,j] = random.randrange(1,6)  

print(t)
print(D)
#dj the delay at each node
d = []
for i in range(N):
    number = random.randrange(0,3)  
    d.append(number)
d[D - 1] = 0
d[0] = 0

print(d)




{(1, 2): 4, (1, 3): 4, (2, 4): 5, (2, 5): 1, (2, 6): 4, (3, 6): 2, (4, 5): 1, (4, 9): 1, (5, 8): 3, (6, 7): 1, (6, 8): 5, (7, 8): 1, (7, 11): 2, (8, 9): 2, (8, 10): 4, (8, 11): 1, (9, 12): 3, (9, 13): 1, (10, 12): 1, (11, 12): 4, (11, 14): 1, (12, 13): 3, (12, 14): 3, (13, 15): 2, (14, 16): 2, (14, 17): 2, (15, 16): 4, (15, 18): 2, (16, 17): 2, (16, 18): 4, (17, 18): 5}
18
[0, 2, 1, 1, 1, 1, 1, 0, 2, 0, 1, 0, 0, 1, 1, 1, 2, 0]


#### Step 3: Define your sets

In [4]:
#number of Nodes
S = N

#### Step 5: Define your decision variables

In [5]:
# objective variables
tplus = m.addVar(vtype = GRB.INTEGER, lb = 0.0, obj= 2, name = "t+")
cplus = m.addVar(vtype = GRB.INTEGER, lb = 0.0, obj = 5, name = "c+")

# normal variables
tminus = m.addVar(vtype =GRB.INTEGER, lb = 0.0, name = "t-")
cminus = m.addVar(vtype = GRB.INTEGER, lb = 0.0, name = "c-")

x = {}
for i in range(S):
    for j in range(S):
        if (i, j) in t:
            x[i,j] = m.addVar(vtype=GRB.BINARY, lb=0.0, name="x_"+str(i)+str(j))

R = m.addVar(vtype=GRB.BINARY)

C = m.addVar(vtype = GRB.INTEGER, lb = 0.0)

Y = m.addVar(vtype = GRB.INTEGER, lb = 0.0)


#### Step 6: Set your objective function

In [6]:
m.modelSense = GRB.MINIMIZE

#### Step 6: Add your constraints

In [10]:
# m.addConstrs((sum(x[i,j] for j in range(J)) <= B[i]*y[i] for i in range(I))) 

m.addConstr(Y == (sum(x[i,j] * (t[i,j] + d[j - 1]) for i in range(S+1) for j in range(S+1) if (i,j) in x) and i != j)) #1

m.addConstr(M*R >= Y-H) #2

m.addQConstr(C == ((1-R)*c + R*c*p*(Y-H))) #3

m.addConstr((C - (cplus + cminus)) == 10) #4

m.addConstr((sum(x[1,j] for j in range(2,S + 1) if (1,j) in x)) == 1) #5

m.addConstr((sum(x[i,D] for i in range(S + 1) if (i,D) in x) and i != D) == 1) #6

for k in range(2, S):
    m.addConstr((sum(x[i,k] for i in range(S + 1) if (i,k) in x and i != k)) == (sum(x[k,j] for j in range(S + 1) if (k,j) in x and j != k))) #7

m.addConstr((Y - (tplus + tminus)) == 20) #8

m.update()


#### Step 7: Solve the model

In [8]:
#m.Params.TimeLimit=300 #(seconds) optional: sets a time limit for optimization only if you need to prematurely stop the solution procedure
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 22 rows, 35 columns and 65 nonzeros
Model fingerprint: 0xad6ebbf0
Model has 1 quadratic constraint
Variable types: 0 continuous, 35 integer (29 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  QMatrix range    [8e+00, 8e+00]
  QLMatrix range   [1e+00, 2e+02]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [4e+00, 4e+00]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


#### Step 8: Print variable values  (The Messy Way)

In [9]:
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.x))
    
#https://docs.python.org/2.4/lib/typesseq-strings.html

AttributeError: Unable to retrieve attribute 'x'

#### Step 8 Alternate: Print the solution (The Easy To Read Way)

In [ ]:
print('\nTOTAL COSTS: %g' % m.objVal) #gets the objective function value
print('SOLUTION:')
for i in range(I):
    if y[i].x > 0.99:   #chooses the binary y variables which get the value of 1
        print('Facility %s open' % i)
        for j in range(J):
            if x[i, j].x > 0:  #chooses the continuous x variables which get a nonzero value
                print('  ships %g thousand units to customer %s' %
                      (x[i,j].x, j))
    else:
        print('Facility %s closed!' % i)

#### Optional Step: Exporting the LP File for Debugging
Do you think that something is wrong with your model? Export it to an LP file and check if it is in the desired shape.

In [ ]:
m.write("checkModel.lp")